In [43]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer # type: ignore
from tensorflow.keras.preprocessing.sequence import pad_sequences #type: ignore
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import wordcloud
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ahmad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [44]:
tokenizer = Tokenizer(num_words = 30730, oov_token="<OOV>")
#tokenizer.fit_on_texts(sentences)
#word_index = tokenizer.word_index
#print(word_index)

In [45]:
df = pd.read_csv("data/tweets.csv")
df = df.set_index("id")
df.head()

keyword        location                                               text  \
id                                                                              
0   ablaze             NaN  Communal violence in Bhainsa, Telangana. "Ston...   
1   ablaze             NaN  Telangana: Section 144 has been imposed in Bha...   
2   ablaze   New York City  Arsonist sets cars ablaze at dealership https:...   
3   ablaze  Morgantown, WV  Arsonist sets cars ablaze at dealership https:...   
4   ablaze             NaN  "Lord Jesus, your love brings freedom and pard...   

    target  
id          
0        1  
1        1  
2        1  
3        1  
4        0

<h1>balancing the data</h1>

In [46]:
target = df["target"].value_counts()
plt.pie([target[1],target[0]],labels=["Disastrous","Non Disastrours"])
plt.show()

In [47]:
print(target[1])
zerorows = df[df["target"]==0]
onerows = df[df["target"]==1]
zerorows = zerorows.iloc[:2114]
new_df = pd.concat([zerorows,onerows])


2114


In [48]:
target = new_df["target"].value_counts()
plt.pie([target[1],target[0]],labels=["Disastrous","Non Disastrours"])
plt.show()

In [49]:
print(new_df.describe())

            target
count  4228.000000
mean      0.500000
std       0.500059
min       0.000000
25%       0.000000
50%       0.500000
75%       1.000000
max       1.000000


<h1>Removing URLs from data using regular expression</h1>

In [50]:
new_df.sort_index()


keyword        location  \
id                               
0       ablaze             NaN   
1       ablaze             NaN   
2       ablaze   New York City   
3       ablaze  Morgantown, WV   
4       ablaze             NaN   
...        ...             ...   
11338  wrecked         Puchong   
11354  wrecked             NaN   
11355  wrecked          Recife   
11359  wrecked  Washington, DC   
11369  wrecked             NaN   

                                                    text  target  
id                                                                
0      Communal violence in Bhainsa, Telangana. "Ston...       1  
1      Telangana: Section 144 has been imposed in Bha...       1  
2      Arsonist sets cars ablaze at dealership https:...       1  
3      Arsonist sets cars ablaze at dealership https:...       1  
4      "Lord Jesus, your love brings freedom and pard...       0  
...                                                  ...     ...  
11338  Kesian ular. We have wrecked their natural hab...       1  
11354  Yeah, proper Liverpool fans wrecked Man City’s...       1  
11355  "Trump and Sisi 'rejected foreign exploitation...       1  
11359  "Trump and Sisi 'rejected foreign exploitation...       1  
11369     Jake Corway wrecked while running 14th at IRP.       1  

[4228 rows x 4 columns]

In [53]:
url_pattren = re.compile(r'https?://\S+|www\.\S+')
for i in range(len(new_df)):
    a = new_df["text"].iloc[i]
    b = url_pattren.findall(a)
    if not b:
        continue
    else:
        a = a.replace(b[0],"")
        new_df["text"].iloc[i] = a
    

C:\Users\Ahmad\AppData\Local\Temp\ipykernel_15192\1947969119.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["text"].iloc[i] = a
C:\Users\Ahmad\AppData\Local\Temp\ipykernel_15192\1947969119.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["text"].iloc[i] = a


<h1>Stemming and Tokenization</h1>

In [54]:
stemmer = PorterStemmer()

def stem_text(text):
    words = word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

new_df["text"] = new_df["text"].apply(stem_text)

print(new_df)

       keyword        location  \
id                               
4       ablaze             NaN   
5       ablaze              OC   
8       ablaze    Accra, Ghana   
9       ablaze       Searching   
11      ablaze             NaN   
...        ...             ...   
11338  wrecked         Puchong   
11354  wrecked             NaN   
11355  wrecked          Recife   
11359  wrecked  Washington, DC   
11369  wrecked             NaN   

                                                    text  target  
id                                                                
4      `` lord jesu , your love bring freedom and par...       0  
5      if thi child wa chines , thi tweet would have ...       0  
8      nation secur minist , kan dapaah 's side chic ...       0  
9      thi creatur who ’ s soul is no longer clarent ...       0  
11     social media went banana after chuba hubbard a...       0  
...                                                  ...     ...  
11338  kesian ular . 

In [11]:
#tokenizer.fit_on_texts(new_df["text"])

#wl = tokenizer.word_index

#print(wl)

{'<OOV>': 1, 'the': 2, 'in': 3, 'a': 4, 'to': 5, 'of': 6, 'and': 7, 'is': 8, 'i': 9, 'on': 10, '’': 11, 'for': 12, 'that': 13, 'it': 14, 'thi': 15, 'at': 16, 'you': 17, 'wa': 18, 'with': 19, 'have': 20, 'are': 21, "'s": 22, 'by': 23, 'from': 24, 'an': 25, 'be': 26, 's': 27, 'we': 28, 'ha': 29, 'fire': 30, 'my': 31, 'as': 32, 'after': 33, 't': 34, 'peopl': 35, 'not': 36, 'they': 37, '…': 38, 'been': 39, 'just': 40, 'amp': 41, 'all': 42, 'up': 43, 'but': 44, 'hi': 45, 'he': 46, 'attack': 47, 'bomb': 48, 'so': 49, 'kill': 50, 'who': 51, 'there': 52, 'were': 53, 'like': 54, 'http': 55, 'one': 56, 'no': 57, 'when': 58, 'about': 59, 'burn': 60, 'bodi': 61, 'will': 62, 'do': 63, 'if': 64, 'year': 65, 'can': 66, 'co': 67, 'iran': 68, 'bag': 69, 'out': 70, 'me': 71, 'what': 72, 'now': 73, 'more': 74, 'new': 75, '1': 76, 'over': 77, 'our': 78, 'down': 79, 'say': 80, 'two': 81, "n't": 82, 'their': 83, 'get': 84, "''": 85, 'australia': 86, 'or': 87, 'us': 88, '“': 89, 'how': 90, 'your': 91, 'had':

In [55]:
new_df2 = new_df.sample(frac=1,random_state=42)
new_df2 = new_df.drop(["keyword","location"],axis=1)

new_df2.describe()

target
count  4228.000000
mean      0.500000
std       0.500059
min       0.000000
25%       0.000000
50%       0.500000
75%       1.000000
max       1.000000

In [56]:
training_Sentences = new_df2[["text","target"]].iloc[:3381]
testing_Sentences = new_df2[["text","target"]].iloc[3381:]


tokenizer.fit_on_texts(training_Sentences["text"])
wl = tokenizer.word_index

training_Sequences = tokenizer.texts_to_sequences(training_Sentences["text"])
testing_Sequences = tokenizer.texts_to_sequences(testing_Sentences["text"])
padded_train = pad_sequences(training_Sequences,padding="post")
padded_test = pad_sequences(testing_Sequences,padding="post")
dfcsv = pd.DataFrame(padded_train)
dfcsv.to_csv("padded.csv")

In [58]:
training_Sentences = new_df2[["text","target"]].iloc[:3381]
testing_Sentences = new_df2[["text","target"]].iloc[3381:]

vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(training_Sentences["text"])
y = vectorizer.fit_transform(testing_Sentences["text"])

In [59]:

nbmodel = GaussianNB()

nbmodel.fit(x.toarray(),training_Sentences["target"])

y_pred = nbmodel.predict(y.toarray())

print(classification_report(testing_Sentences["target"],y_pred))

ValueError: X has 3040 features, but GaussianNB is expecting 8202 features as input.

In [ ]:
testing_Sequences = tokenizer.texts_to_sequences(testing_Sentences)
padded_test = pad_sequences(testing_Sequences)